In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%pip install ultralytics
%pip install mediapipe
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.1/225.8 GB disk)


In [ ]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import torch

In [ ]:

# mediapipe 손 감지 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

best_model = '/content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/best.pt'
# best_model = 'best.pt'


# YOLO 객체 감지 모델 초기화
yolo_model = YOLO(best_model)

prev_boxes = []

cap = cv2.VideoCapture(1) # for Mac
# cap = cv2.VideoCapture(0) # for Windows

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR 이미지를 RGB로 변환
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(frame, 1)
    results = hands.process(image)

    # YOLO 객체 감지
    bboxes = yolo_model.detect_objects(image)


    # 객체 감지 결과를 이미지에 표시
    if bboxes.shape[0] > 0:
      prev_boxes = bboxes
      for bbox in bboxes:
          x1, y1, x2, y2 = bbox
          cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    else:
      if prev_boxes:
        for prev_box in prev_boxes:
            for box in bboxes:
                iou = (min(prev_box[2], box[2]) - max(prev_box[0], box[0])) * \
                      (min(prev_box[3], box[3]) - max(prev_box[1], box[1])) / \
                      float((prev_box[2] - prev_box[0]) * (prev_box[3] - prev_box[1]) +
                            (box[2] - box[0]) * (box[3] - box[1]))

                if iou > 0.5:
                    cv2.rectangle(frame, (int(prev_box[0]), int(prev_box[1])),
                                  (int(prev_box[2]), int(prev_box[3])), (0, 255, 0), 2)
                    break

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
              with torch.no_grad():
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)


    cv2.imshow('Frame', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()